In [106]:
import numpy as np
import pandas as pd
import math
import Bio.PDB
from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import is_aa
from Bio.PDB.Polypeptide import three_to_one
import json
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import os
import pickle

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [77]:
df = pd.read_pickle('df_all_seqfeature.pkl')

In [78]:
df

,Sequence,Symmetry,Qs_state,Qs_type,aa_composition,entropy,dipeptide_composition
0,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,C6,monomer,homomer,"[102, 24, 12, 36, 0, 30, 84, 66, 72, 54, 108, ...",3.941791,"[6, 0, 0, 12, 0, 12, 12, 6, 0, 12, 12, 12, 6, ..."
1,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,C2,monomer,homomer,"[34, 26, 22, 20, 0, 10, 16, 22, 2, 20, 30, 26,...",4.049619,"[8, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 8, 0, 0, 0, ..."
2,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,C6,monomer,homomer,"[108, 24, 12, 36, 0, 30, 84, 66, 66, 54, 108, ...",3.941942,"[6, 0, 0, 12, 0, 12, 12, 12, 0, 12, 12, 12, 6,..."
3,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTSLDAAKSELDKA...,C2,monomer,homomer,"[32, 26, 20, 22, 0, 10, 16, 22, 2, 20, 30, 24,...",4.039130,"[8, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 8, 0, 0, 0, ..."
4,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,C6,monomer,homomer,"[108, 24, 12, 36, 0, 30, 84, 66, 66, 54, 108, ...",3.941942,"[6, 0, 0, 12, 0, 12, 12, 12, 0, 12, 12, 12, 6,..."
...,...,...,...,...,...,...,...
105453,EVQLLESGGGLVQPGGSLRLSCAASGFTFSYYYMQWVRQAPGKGLE...,C2,dodecamer,hetromer,"[10, 6, 8, 7, 4, 7, 5, 18, 2, 4, 14, 8, 3, 7, ...",4.057629,"[1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, ..."
105454,AVCPGTLNRCEVVMGNLEIVLTGHNADLSFLQWIREVTGYVLVAMN...,C2,dodecamer,hetromer,"[23, 30, 32, 27, 45, 17, 26, 49, 20, 22, 43, 2...",4.189246,"[0, 0, 1, 1, 4, 1, 3, 2, 2, 1, 2, 1, 1, 0, 1, ..."
105455,EVQLLESGGGLVQPGGSLRLSCAASGFTFSYYYMQWVRQAPGKGLE...,C2,dodecamer,hetromer,"[13, 6, 8, 6, 4, 7, 5, 23, 2, 3, 18, 7, 3, 7, ...",3.986567,"[2, 1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1, 1, ..."
105456,EVQLLESGGGLVQPGGSLRLSCAASGFTFSYYYMQWVRQAPGKGLE...,C2,dodecamer,hetromer,"[11, 6, 8, 7, 4, 7, 5, 19, 2, 3, 15, 8, 3, 7, ...",4.048984,"[1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, ..."


In [79]:
features = []
for i in tqdm(range(len(df))):
    features.append(df['aa_composition'][i] + [df['entropy'][i]] + df['dipeptide_composition'][i])
features = np.array(features)

100%|██████████| 105458/105458 [00:01<00:00, 54203.26it/s]


In [80]:
symmetry = df['Symmetry']
qs = df['Qs_state']
qt = df['Qs_type']

In [81]:
def assign_others(arr, threshold):
    arr = list(arr)
    from collections import Counter
    counter = dict(Counter(arr))

    remove_entries = []
    for key in counter:
        if counter[key] <= threshold:
            remove_entries.append(key)
    
    for i in range(len(arr)):
        if arr[i] in remove_entries:
            arr[i] = 'others'
    
    return arr


In [82]:
threshold = 30
symmetry = assign_others(symmetry, threshold)
qs = assign_others(qs, threshold)
qt = assign_others(qt, threshold)

In [83]:
#np.unique(symmetry, return_counts = True), np.unique(qs, return_counts = True), np.unique(qt, return_counts = True)

In [84]:
le1 = LabelEncoder()
symmetry = le1.fit_transform(symmetry)
le2 = LabelEncoder()
qs = le2.fit_transform(qs)
le3 = LabelEncoder()
qt = le3.fit_transform(qt)

In [130]:
print(le1.classes_)
print(le2.classes_)
print(le3.classes_)

array(['C1', 'C2', 'C2,C2', 'C3', 'C4', 'C5', 'C6', 'D2', 'D3', 'D4',
       'D5', 'D6', 'others'], dtype='<U6')

## Train-Test Split

In [86]:
from sklearn.model_selection import train_test_split

In [87]:
x1_train, x1_test, y1_train, y1_test = train_test_split(features, symmetry, test_size=0.2, random_state=42, shuffle=True, stratify=symmetry)
x2_train, x2_test, y2_train, y2_test = train_test_split(features, qs, test_size=0.2, random_state=42, shuffle=True, stratify=qs)
x3_train, x3_test, y3_train, y3_test = train_test_split(features, qt, test_size=0.2, random_state=42, shuffle=True, stratify=qt)


### Storing Results

In [88]:
path_to_results = 'results'
if not os.path.exists(path_to_results):
    os.makedirs(path_to_results)
    os.makedirs(os.path.join(path_to_results, 'symmetry'))
    os.makedirs(os.path.join(path_to_results, 'qs'))
    os.makedirs(os.path.join(path_to_results, 'qt'))
    

In [89]:
dirs = os.listdir(path_to_results)
for i in range(len(dirs)):
    if os.listdir(os.path.join(path_to_results, dirs[i])) == []:
        os.makedirs(os.path.join(os.path.join(path_to_results, dirs[i]), 'models'))
        os.makedirs(os.path.join(os.path.join(path_to_results, dirs[i]), 'metrics'))

### Classifier 1 : Logistic Regression

In [111]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import matthews_corrcoef, make_scorer
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [ ]:
def

In [122]:
model = LogisticRegression(random_state=1)
params = {'penalty' : ['l2'], 'C' : [0.01,0.1,1], 'class_weight' : [None, 'balanced'], 'multi_class' : ['auto']}
#params = {'penalty' : ['l2'], 'C' : [0.1], 'class_weight' : [None], 'multi_class' : ['auto']}
scoring = make_scorer(matthews_corrcoef)
n_jobs = -1
refit = True
cv = 5

model_type = 'lr'
model_name = 'Logistic Regression'

In [123]:
print('Training ' + model_name + ' For Symmetry')
lr1 = GridSearchCV(estimator=model, param_grid=params, scoring=scoring, n_jobs=n_jobs, refit=refit, cv=cv)
lr1.fit(x1_train, y1_train)
print('Training ' + model_name + ' For Quaternary State')
lr2 = GridSearchCV(estimator=model, param_grid=params, scoring=scoring, n_jobs=n_jobs, refit=refit, cv=cv)
lr2.fit(x2_train, y2_train)
print('Training ' + model_name + ' For Quaternary Type')
lr3 = GridSearchCV(estimator=model, param_grid=params, scoring=scoring, n_jobs=n_jobs, refit=refit, cv=cv)
lr3.fit(x3_train, y3_train)

Training Logisitic Regression For Symmetry


/home/mtech/.conda/envs/sushumna/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mtech/.conda/envs/sushumna/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

Training Logisitic Regression For Quaternary State


/home/mtech/.conda/envs/sushumna/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mtech/.conda/envs/sushumna/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

Training Logisitic Regression For Quaternary Type


/home/mtech/.conda/envs/sushumna/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mtech/.conda/envs/sushumna/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=1), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1],
                         'class_weight': [None, 'balanced'],
                         'multi_class': ['auto'], 'penalty': ['l2']},
             scoring=make_scorer(matthews_corrcoef))

In [125]:
print('{} Model Results'.format(model_name))
print('\n')

print('Cross-validation Results')
print('\n')
print('Symmetry Prediction')
print('Best Model is {}'.format(lr1.best_estimator_))
print('Crossvalidation score is {}'.format(lr1.best_score_))
print('Quaternary State Prediction')
print('Best Model is {}'.format(lr2.best_estimator_))
print('Crossvalidation score is {}'.format(lr2.best_score_))
print('Quaternary Type Prediction')
print('Best Model is {}'.format(lr3.best_estimator_))
print('Crossvalidation score is {}'.format(lr3.best_score_))

with open('results/symmetry/models/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr1, f)
with open('results/qs/models/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr2, f)
with open('results/qt/models/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr3, f)

y1_pred = lr1.predict(x1_test)
y2_pred = lr2.predict(x2_test)
y3_pred = lr3.predict(x3_test)

def metrics(y_true, y_pred):
    metrics_dict = {}
    metrics_dict['f1'] = f1_score(y_true, y_pred, average = None)
    metrics_dict['macro_f1'] = f1_score(y_true, y_pred, average='macro')
    metrics_dict['micro_f1'] = f1_score(y_true, y_pred, average='micro')
    metrics_dict['weighted_f1'] = f1_score(y_true, y_pred, average='weighted')
    metrics_dict['acc'] = accuracy_score(y_true, y_pred)
    metrics_dict['mcc'] = matthews_corrcoef(y_true, y_pred)
    return metrics_dict

lr1_metrics = metrics(y1_test, y1_pred)
lr2_metrics = metrics(y2_test, y2_pred)
lr3_metrics = metrics(y3_test, y3_pred)

with open('results/symmetry/metrics/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr1_metrics, f)
with open('results/qs/metrics/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr2_metrics, f)
with open('results/qt/metrics/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr3_metrics, f)

print('\n')
print('Results on Test Data')
print('\n')
print('For Symmetry Prediction')
print('Accuracy = {}'.format(lr1_metrics['acc']))
print('MCC = {}'.format(lr1_metrics['mcc']))
print('Class wise F1 = {}'.format(lr1_metrics['f1']))
print('Macro F1 = {}'.format(lr1_metrics['macro_f1']))
print('Micro F1 = {}'.format(lr1_metrics['micro_f1']))
print('Weighted F1 = {}'.format(lr1_metrics['weighted_f1']))

print('\n')
print('For Quaternary State Prediction')
print('Accuracy = {}'.format(lr2_metrics['acc']))
print('MCC = {}'.format(lr2_metrics['mcc']))
print('Class wise F1 = {}'.format(lr2_metrics['f1']))
print('Macro F1 = {}'.format(lr2_metrics['macro_f1']))
print('Micro F1 = {}'.format(lr2_metrics['micro_f1']))
print('Weighted F1 = {}'.format(lr2_metrics['weighted_f1']))

print('\n')
print('For Quaternary Type Prediction')
print('Accuracy = {}'.format(lr3_metrics['acc']))
print('MCC = {}'.format(lr3_metrics['mcc']))
print('Class wise F1 = {}'.format(lr3_metrics['f1']))
print('Macro F1 = {}'.format(lr3_metrics['macro_f1']))
print('Micro F1 = {}'.format(lr3_metrics['micro_f1']))
print('Weighted F1 = {}'.format(lr3_metrics['weighted_f1']))


Logistic Regression Model Results


Cross-validation Results


Symmetry Prediction
Best Model is LogisticRegression(C=0.1, random_state=1)
Crossvalidation score is 0.24727268553103485
Quaternary State Prediction
Best Model is LogisticRegression(C=0.01, random_state=1)
Crossvalidation score is 0.12852735884724048
Quaternary Type Prediction
Best Model is LogisticRegression(C=0.01, class_weight='balanced', random_state=1)
Crossvalidation score is 0.32134013414020257


Results on Test Data


For Symmetry Prediction
Accuracy = 0.506874644414944
MCC = 0.24483054824424888
Class wise F1 = [0.54838172 0.6009102  0.09837587 0.21491914 0.         0.
 0.44247788 0.23376623 0.21069182 0.21538462 0.05479452 0.17647059
 0.29411765]
Macro F1 = 0.23771463319400074
Micro F1 = 0.506874644414944
Weighted F1 = 0.4602342413865662


For Quaternary State Prediction
Accuracy = 0.20254124786648967
MCC = 0.02484136990693905
Class wise F1 = [0.03243881 0.42760587 0.00273598 0.02336697 0.         0.
 0.         0.

### Classsifier 2: Decision Tree

In [126]:
from sklearn.tree import DecisionTreeClassifier

In [127]:
model = DecisionTreeClassifier(random_state=1)
params = {'max_depth' : [10, 20, 40, -1], 'min_samples_split' : [16, 32], 'class_weight' : [None, 'balanced']}
#params = {'penalty' : ['l2'], 'C' : [0.1], 'class_weight' : [None], 'multi_class' : ['auto']}
scoring = make_scorer(matthews_corrcoef)
n_jobs = -1
refit = True
cv = 5

model_type = 'dt'
model_name = 'Decision Tree'

In [128]:
print('Training ' + model_name + ' For Symmetry')
lr1 = GridSearchCV(estimator=model, param_grid=params, scoring=scoring, n_jobs=n_jobs, refit=refit, cv=cv)
lr1.fit(x1_train, y1_train)
print('Training ' + model_name + ' For Quaternary State')
lr2 = GridSearchCV(estimator=model, param_grid=params, scoring=scoring, n_jobs=n_jobs, refit=refit, cv=cv)
lr2.fit(x2_train, y2_train)
print('Training ' + model_name + ' For Quaternary Type')
lr3 = GridSearchCV(estimator=model, param_grid=params, scoring=scoring, n_jobs=n_jobs, refit=refit, cv=cv)
lr3.fit(x3_train, y3_train)

Training Decision Tree For Symmetry
Training Decision Tree For Quaternary State
Training Decision Tree For Quaternary Type


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=1), n_jobs=-1,
             param_grid={'class_weight': [None, 'balanced'],
                         'max_depth': [10, 20, 40, -1],
                         'min_samples_split': [16, 32]},
             scoring=make_scorer(matthews_corrcoef))

In [129]:
print('{} Model Results'.format(model_name))
print('\n')

print('Cross-validation Results')
print('\n')
print('Symmetry Prediction')
print('Best Model is {}'.format(lr1.best_estimator_))
print('Crossvalidation score is {}'.format(lr1.best_score_))
print('Quaternary State Prediction')
print('Best Model is {}'.format(lr2.best_estimator_))
print('Crossvalidation score is {}'.format(lr2.best_score_))
print('Quaternary Type Prediction')
print('Best Model is {}'.format(lr3.best_estimator_))
print('Crossvalidation score is {}'.format(lr3.best_score_))

with open('results/symmetry/models/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr1, f)
with open('results/qs/models/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr2, f)
with open('results/qt/models/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr3, f)

y1_pred = lr1.predict(x1_test)
y2_pred = lr2.predict(x2_test)
y3_pred = lr3.predict(x3_test)

def metrics(y_true, y_pred):
    metrics_dict = {}
    metrics_dict['f1'] = f1_score(y_true, y_pred, average = None)
    metrics_dict['macro_f1'] = f1_score(y_true, y_pred, average='macro')
    metrics_dict['micro_f1'] = f1_score(y_true, y_pred, average='micro')
    metrics_dict['weighted_f1'] = f1_score(y_true, y_pred, average='weighted')
    metrics_dict['acc'] = accuracy_score(y_true, y_pred)
    metrics_dict['mcc'] = matthews_corrcoef(y_true, y_pred)
    return metrics_dict

lr1_metrics = metrics(y1_test, y1_pred)
lr2_metrics = metrics(y2_test, y2_pred)
lr3_metrics = metrics(y3_test, y3_pred)

with open('results/symmetry/metrics/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr1_metrics, f)
with open('results/qs/metrics/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr2_metrics, f)
with open('results/qt/metrics/{}.pkl'.format(model_type), 'wb') as f:
    pickle.dump(lr3_metrics, f)

print('\n')
print('Results on Test Data')
print('\n')
print('For Symmetry Prediction')
print('Accuracy = {}'.format(lr1_metrics['acc']))
print('MCC = {}'.format(lr1_metrics['mcc']))
print('Class wise F1 = {}'.format(lr1_metrics['f1']))
print('Macro F1 = {}'.format(lr1_metrics['macro_f1']))
print('Micro F1 = {}'.format(lr1_metrics['micro_f1']))
print('Weighted F1 = {}'.format(lr1_metrics['weighted_f1']))

print('\n')
print('For Quaternary State Prediction')
print('Accuracy = {}'.format(lr2_metrics['acc']))
print('MCC = {}'.format(lr2_metrics['mcc']))
print('Class wise F1 = {}'.format(lr2_metrics['f1']))
print('Macro F1 = {}'.format(lr2_metrics['macro_f1']))
print('Micro F1 = {}'.format(lr2_metrics['micro_f1']))
print('Weighted F1 = {}'.format(lr2_metrics['weighted_f1']))

print('\n')
print('For Quaternary Type Prediction')
print('Accuracy = {}'.format(lr3_metrics['acc']))
print('MCC = {}'.format(lr3_metrics['mcc']))
print('Class wise F1 = {}'.format(lr3_metrics['f1']))
print('Macro F1 = {}'.format(lr3_metrics['macro_f1']))
print('Micro F1 = {}'.format(lr3_metrics['micro_f1']))
print('Weighted F1 = {}'.format(lr3_metrics['weighted_f1']))


Decision Tree Model Results


Cross-validation Results


Symmetry Prediction
Best Model is DecisionTreeClassifier(max_depth=40, min_samples_split=16, random_state=1)
Crossvalidation score is 0.6024444587655579
Quaternary State Prediction
Best Model is DecisionTreeClassifier(max_depth=40, min_samples_split=16, random_state=1)
Crossvalidation score is 0.4741647576977668
Quaternary Type Prediction
Best Model is DecisionTreeClassifier(max_depth=40, min_samples_split=16, random_state=1)
Crossvalidation score is 0.6398511117084835


Results on Test Data


For Symmetry Prediction
Accuracy = 0.7379575194386497
MCC = 0.6303382094437369
Class wise F1 = [0.77539813 0.78346291 0.58042145 0.73687869 0.39705882 0.56626506
 0.65030675 0.6292316  0.64025696 0.4375     0.56666667 0.51282051
 0.55555556]
Macro F1 = 0.6024479312756009
Micro F1 = 0.7379575194386497
Weighted F1 = 0.7361099285713737


For Quaternary State Prediction
Accuracy = 0.18808078892471078
MCC = 0.04149388380344337
Class wise F1 = [0